<B>THIS IS DATA SCIENCE CAPSTONE PROJECT</B>

In [7]:
!pip install geocoder
!pip install folium
!pip install requests
!pip install geopy
import requests
import json
import xml
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import geocoder
import folium
import parser
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
%matplotlib inline
import warnings

In [8]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')
wiki_data.head()

[<meta charset="utf-8"/>,
 <title>List of postal codes of Canada: M - Wikipedia</title>,
 <script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XsnThApAIC4AAE50mMAAAAAO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":958430791,"wgRevisionId":958430791,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRe

In [9]:
tr = BeautifulSoup(extracting_data, 'lxml')

# Converting content of PostalCode HTML table as dataframe

column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text.strip('\n')
            i = i + 1
        elif i == 1:
            borough = td.text.strip('\n')
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

In [10]:
toronto.head()

,Postalcode,Borough,Neighborhood
0,0,0,0
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [11]:
mask= toronto['Neighborhood'] == ''
t=toronto[mask]
t

,Postalcode,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
8,M8A,Not assigned,
11,M2B,Not assigned,
16,M7B,Not assigned,
17,M8B,Not assigned,
20,M2C,Not assigned,
25,M7C,Not assigned,
26,M8C,Not assigned,
29,M2E,Not assigned,


In [12]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!=0]
toronto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 3 to 179
Data columns (total 3 columns):
Postalcode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 3.2+ KB


In [13]:
mask= toronto['Neighborhood'] == 'Not assigned'
t=toronto[mask]
t

,Postalcode,Borough,Neighborhood


In [14]:
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1

In [15]:
t1 = toronto.groupby('Neighborhood')['Neighborhood'].count() == 2 
t1

Neighborhood
Agincourt                                                                                                        False
Alderwood, Long Branch                                                                                           False
Bathurst Manor, Wilson Heights, Downsview North                                                                  False
Bayview Village                                                                                                  False
Bedford Park, Lawrence Manor East                                                                                False
Berczy Park                                                                                                      False
Birch Cliff, Cliffside West                                                                                      False
Brockton, Parkdale Village, Exhibition Place                                                                     False
Business reply mail Processing Cent

In [17]:
mask= toronto['Postalcode'] == 'M5A'
t=toronto[mask]
t

,Postalcode,Borough,Neighborhood
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [18]:
toronto.describe()

,Postalcode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M7Y,North York,Downsview
freq,1,24,4


In [19]:
df = toronto.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 3 to 179
Data columns (total 3 columns):
Postalcode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 3.2+ KB


In [21]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [22]:
print(df2.shape)
df2.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [23]:
def get_latilong(postal_code):
    
    lati_long_coords = None
    while(lati_long_coords is None):
        
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M5A')

[43.65096410900003, -79.35304116399999]

In [24]:
sf_map = folium.Map(location = [43.65096410900003, -79.35304116399999],zoom_start=12)
sf_map

In [26]:
postal_codes = toronto['Postalcode']    
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [28]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
toronto['Latitude'] = df_coords['Latitude']
toronto['Longitude'] = df_coords['Longitude']

In [31]:
toronto[toronto.Postalcode == 'M5A']

,Postalcode,Borough,Neighborhood,Latitude,Longitude
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.667481,-79.528953


In [32]:
toronto.head(12)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.723265,-79.451211
4,M4A,North York,Victoria Village,43.661790,-79.389390
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.667481,-79.528953
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.808626,-79.189913
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.748900,-79.357220
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.657491,-79.377529
10,M1B,Scarborough,"Malvern, Rouge",43.707279,-79.447500
12,M3B,North York,Don Mills,43.785779,-79.157368
13,M4B,East York,"Parkview Hill, Woodbine Gardens",43.722143,-79.352023
14,M5B,Downtown Toronto,"Garden District, Ryerson",43.689740,-79.308507


In [33]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto['Borough'].unique()),
        toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.
